# Topic Modeling 

### Preprocessing

In [27]:
from sklearn.decomposition import NMF
import pandas as pd, numpy as np, lda, warnings
import pyLDAvis
import pyLDAvis.graphlab
from collections import Counter, defaultdict
from itertools import islice
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from IPython.core.display import HTML
HTML("<style>.container { width:95% !important; }</style>")
warnings.filterwarnings('ignore')

In [28]:
pyLDAvis.enable_notebook()

In [3]:
df_labeled = pd.read_csv('data/df_all.csv', na_values=['?'])
df_labeled.dropna(inplace = True)
df_labeled.head()

,verify_relationship,chemical_name,disease_name,all_info
0,yes_direct,5-HT,psychotic disorders,animal models considered reflect positive symp...
1,no_relation,D-penicillamine,localized scleroderma,case reports patients severe extensive localiz...
2,no_relation,yohimbine,affective disorders,method six patients either obsessive compulsiv...
3,no_relation,calcium,muscle spasms,severe hypokalemia may cause muscle weakness s...
4,no_relation,dexrazoxane,hematologic toxicity,clinical trials patients brain metastases comb...


In [4]:
#Extend all the words in all_info into a list. 
all_words = []
for doc in df_labeled.all_info:
    all_words.extend(doc.split())

In [5]:
#Create a vocabulary
vocab = set(all_words)

In [6]:
len(vocab)

18971

In [7]:
#Create a dictionary of stemmed words and count of the true words.
stem = PorterStemmer()
vocab_dict = defaultdict(list)
for word in vocab: 
    vocab_dict[stem.stem(word)].append((word,all_words.count(word)))
#view first 20 items. 
list(islice(vocab_dict.items(), 20))

[(u'andresist', [('andresistant', 1)]),
 (u'highdens', [('highdensity', 47)]),
 (u'dosageconvers', [('dosageconversion', 2)]),
 (u'telmisartan', [('telmisartan', 29)]),
 (u'polypeptid', [('polypeptide', 2)]),
 (u'four', [('four', 309)]),
 (u'marfan', [('marfan', 16)]),
 (u'oxygensens', [('oxygensensing', 8)]),
 (u'nanoccurr', [('nanoccurrence', 1)]),
 (u'assaysnannk', [('assaysnannk', 5)]),
 (u'putamen', [('putamen', 1)]),
 (u'hyperoxid', [('hyperoxidized', 2), ('hyperoxidation', 15)]),
 (u'cellsnangastr', [('cellsnangastric', 1)]),
 (u'hyperoxia', [('hyperoxia', 1)]),
 (u'nanposterior', [('nanposterior', 1)]),
 (u'infectionnanadenoviru', [('infectionnanadenovirus', 1)]),
 (u'deathnaningest', [('deathnaningestion', 5)]),
 (u'microgramsl', [('microgramsl', 4)]),
 (u'dithian', [('dithianes', 7), ('dithiane', 5)]),
 (u'digit', [('digit', 4), ('digitalization', 2)])]

In [8]:
#key = stemmed word... value = most likely true word
final_vocab_dict = {}
for k,v in vocab_dict.items():
    final_vocab_dict[k] = max(v, key=lambda x:x[1])[0]
#view first 20 items. 
list(islice(final_vocab_dict.items(), 20))

[(u'andresist', 'andresistant'),
 (u'highdens', 'highdensity'),
 (u'dosageconvers', 'dosageconversion'),
 (u'telmisartan', 'telmisartan'),
 (u'polypeptid', 'polypeptide'),
 (u'four', 'four'),
 (u'marfan', 'marfan'),
 (u'crisisdescrib', 'crisisdescribe'),
 (u'oxygensens', 'oxygensensing'),
 (u'nanoccurr', 'nanoccurrence'),
 (u'assaysnannk', 'assaysnannk'),
 (u'putamen', 'putamen'),
 (u'hyperoxid', 'hyperoxidation'),
 (u'cellsnangastr', 'cellsnangastric'),
 (u'hyperoxia', 'hyperoxia'),
 (u'nanposterior', 'nanposterior'),
 (u'arterioven', 'arteriovenous'),
 (u'infectionnanadenoviru', 'infectionnanadenovirus'),
 (u'list', 'listed'),
 (u'deathnaningest', 'deathnaningestion')]

In [9]:
#Stemming each word in X. 
stemmed_info = []
for doc in df_labeled['all_info']:
    d = []
    for word in doc.split():
        d.append(str(stem.stem(word)))
    stemmed_info.append(" ".join(d))
df_labeled['stemmed_info'] = stemmed_info

In [10]:
#boolean verify_relationship (merging yes_direct and yes_indirect to simplify our graph)
temp = {'yes_direct': True, 'yes_indirect':True, 'no_relation':False}
df_labeled['boolean_relationship'] = df_labeled['verify_relationship'].map(temp)
df_labeled = df_labeled.drop(['verify_relationship'], axis=1).copy()

In [11]:
df_labeled.head()

,chemical_name,disease_name,all_info,stemmed_info,boolean_relationship
0,5-HT,psychotic disorders,animal models considered reflect positive symp...,anim model consid reflect posit symptom schizo...,True
1,D-penicillamine,localized scleroderma,case reports patients severe extensive localiz...,case report patient sever extens local sclerod...,False
2,yohimbine,affective disorders,method six patients either obsessive compulsiv...,method six patient either obsess compuls disor...,False
3,calcium,muscle spasms,severe hypokalemia may cause muscle weakness s...,sever hypokalemia may caus muscl weak sever hy...,False
4,dexrazoxane,hematologic toxicity,clinical trials patients brain metastases comb...,clinic trial patient brain metastas combin dex...,False


### Vectorize data

In [12]:
X = df_labeled['stemmed_info']
y = df_labeled['boolean_relationship']

In [13]:
vect = CountVectorizer(min_df=1)
tfidf = TfidfVectorizer(min_df=1)

X_vect = vect.fit_transform(X)
X_tfidf = tfidf.fit_transform(X)

In [14]:
train_features_vect = vect.get_feature_names()
train_features_tfidf = tfidf.get_feature_names()
print("len_feature_vect: ", len(train_features_vect))
print("len_feature_tfidf: ", len(train_features_tfidf))

('len_feature_vect: ', 15389)
('len_feature_tfidf: ', 15389)


### NMF (Non-Negative Matrix Factorization)

#### Count Vectorizer

In [18]:
nmf = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5).fit(X_vect)
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
#         print("Topic stemmed #%d:" % topic_idx)
#         print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
#         print()
        print("Topic words #%d:" % topic_idx)
        topic_words = []
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            try:
                #change word back to most likely true word
                topic_words.append(final_vocab_dict[feature_names[i]])
            except:
                topic_words.append(feature_names[i])
        print(" ".join(topic_words))
print_top_words(nmf, train_features_vect, 30)

Topic words #0:
patients treatment study response months therapy disease median days treated dose clinical levels years survival complete evaluated rate one trials total chronic weeks performed hepatitis receiving toxicity observed severe case
Topic words #1:
cells activity cancer inhibition human expression tumor lines increased induced apoptosis growth protein results melanoma potential effects inhibitor pathway showed kinase cytotoxicity treatment lung also erk mice signaling dna study
Topic words #2:
used increased risk study associated may drug clozapine reported children also bodyweight toxicity levobupivacaine less potential due development glucose olanzapine acid disease however risperidone adults profile clinical frequently well treatment
Topic words #3:
mg tumor te dose med ep highgrade gliomas cpt days adults zaleplon activity including system dosage demonstrated study central mgkg malignant derived nude 11 nervous current well produced resistance subjects
Topic words #4:
ef

** Top 3 topics: **
- opiates
- skin cell diseases
- eyes and cats

#### TF-IDF

In [19]:
nmf = NMF(n_components=10, random_state=1, alpha=.1, l1_ratio=.5).fit(X_tfidf)
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        #print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        topic_words = []
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            try:
                #change word back to most likely true word
                topic_words.append(final_vocab_dict[feature_names[i]])
            except:
                topic_words.append(feature_names[i])
        print(" ".join(topic_words))
print_top_words(nmf, train_features_tfidf, 30)

Topic #0:
cells cancer tumor inhibition activity human growth lines apoptosis expression induced effects protein drug combination curcumin inhibitor pathway cytotoxicity breast fu signaling kinase chloroquine autophagy receptor increased carcinoma vitro results
Topic #1:
patients treatment study therapy disease effects aspirin dose group used trials days pain clinical efficacy stroke fentanyl response toxicity drug associated combination compared may treated rate receiving years severe symptoms
Topic #2:
clozapine bodyweight olanzapine risperidone risk children gain antipsychotic glucose quetiapine ziprasidone mania adolescents adults ep increased receptor profile monitoring atypical less frequently hyperprolactinaemia sedation baseline intolerance reported used published toward
Topic #3:
mg te med cpt gliomas highgrade ep tumor 11 adults procarbazine melphalan medulloblastomas sublines ependymomas br rhabdomyosarcoma busulfan derived childhood intracranially dosage mr athymic pr cyclo

** Top 3 topics: **
- diabetes and cancer
- cats and rage
- skin cell diseases

### LDA (Latent Dirichlet Allocation)

#### Count Vectorizer

In [20]:
model = lda.LDA(n_topics=10, n_iter=3000, random_state=1)
model.fit_transform(X_vect) 
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 30
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vect.get_feature_names())[np.argsort(topic_dist)][:-n_top_words:-1]
    top_words = []
    for word in (' '.join(topic_words)).split():
        try:
            #change word back to most likely true word
            top_words.append(final_vocab_dict[str(word)])
        except:
            top_words.append(str(word))
    print('Topic {}: {}'.format(i, ' '.join(top_words)))

Topic 0: patients diabetes study group treatment insulin effects hypertension disease inhibitor therapy type glucose combination blood levels serum metabolic mg cardiovascular control prevention risk vascular reduced drug angiotensin blocker obesity
Topic 1: patients days effects study treatment group pain symptoms score dose used clinical receiving associated infusion morphine severe mg scale rate improved trials weeks efficacy significantly compared therapy placebo daily
Topic 2: effects used may study patients drug associated receptor treatment disease toxicity clinical potential pain also fentanyl development increased evidence risk results data role dose adverse however activity transdermal improved
Topic 3: patients study used case effects treatment increased risk associated development may months years drug dose artery myocardial therapy intake cause vitamin coronary one disease acid acute treated cancer reported
Topic 4: patients treatment therapy used disease case reported ste

** Top 5 Topics:**
- Drug treatments at clinical trials
- Cancer cells
- Effects of chemo drugs
- Diabetes
- Chemicals that affect the brain

**Note: ** Works only with matrix with integer values, so tfidf would not work.

### GraphLab

In [15]:
#create list of dictionaries of word counts of each document.
docs = []
for doc in df_labeled.stemmed_info:
    docs.append(dict(Counter(doc.split())))

In [16]:
docs[0]

{'5-ht': 1,
 '5-htalter': 1,
 '5-htht': 1,
 'addit': 1,
 'alter': 2,
 'analysi': 1,
 'anim': 3,
 'antagonist': 2,
 'antipsychot': 1,
 'associ': 1,
 'clozapin': 1,
 'conduct': 1,
 'consid': 2,
 'correct': 1,
 'damphetamin': 1,
 'damphetamineinduc': 1,
 'disord': 4,
 'disrupt': 1,
 'evid': 2,
 'factor': 1,
 'gene': 1,
 'gener': 1,
 'ht': 6,
 'htr': 1,
 'hyperact': 1,
 'hypothes': 1,
 'induc': 1,
 'inhibit': 1,
 'involv': 2,
 'methamphetamin': 1,
 'methinduc': 2,
 'model': 2,
 'neural': 1,
 'olanzapin': 1,
 'paranoid': 1,
 'pathophysiolog': 2,
 'phencyclidin': 1,
 'posit': 2,
 'prepuls': 1,
 'psychosi': 2,
 'psychot': 4,
 'rat': 2,
 'receptor': 7,
 'reflect': 2,
 'relat': 1,
 'restor': 1,
 'schizophrenia': 4,
 'second': 1,
 'select': 1,
 'serotonerg': 1,
 'serotonin': 1,
 'sever': 1,
 'similar': 1,
 'studi': 1,
 'suggest': 2,
 'suscept': 1,
 'symptom': 3,
 'target': 1,
 'therapeut': 1,
 'therefor': 1,
 'transmiss': 1,
 'type': 1,
 'use': 2}

In [17]:
final_vocab_dict['adequ']

'adequate'

In [18]:
#change stemmed words back to human-readable words. 
new_docs = []

for doc in docs:
    real_counter = defaultdict(int)
    for k,v in doc.items():
        try:
            real_counter[final_vocab_dict[k]] = v
        except:
            real_counter[k] = v
    new_docs.append(dict(real_counter))
new_docs[0]

{'5-ht': 1,
 '5-htaltered': 1,
 '5-htht': 1,
 'addition': 1,
 'alterations': 2,
 'analysis': 1,
 'animals': 3,
 'antagonist': 2,
 'antipsychotic': 1,
 'associated': 1,
 'clozapine': 1,
 'conducted': 1,
 'considered': 2,
 'correction': 1,
 'damphetamine': 1,
 'damphetamineinduced': 1,
 'disorder': 4,
 'disruption': 1,
 'evidence': 2,
 'factor': 1,
 'gene': 1,
 'generally': 1,
 'ht': 6,
 'htr': 1,
 'hyperactivity': 1,
 'hypothesized': 1,
 'induced': 1,
 'inhibition': 1,
 'involved': 2,
 'methamphetamine': 1,
 'methinduced': 2,
 'models': 2,
 'neural': 1,
 'olanzapine': 1,
 'paranoid': 1,
 'pathophysiology': 2,
 'phencyclidine': 1,
 'positive': 2,
 'prepulse': 1,
 'psychosis': 2,
 'psychotic': 4,
 'rats': 2,
 'receptor': 7,
 'reflecting': 2,
 'related': 1,
 'restored': 1,
 'schizophrenia': 4,
 'second': 1,
 'selective': 1,
 'serotonergic': 1,
 'serotonin': 1,
 'severe': 1,
 'similar': 1,
 'study': 1,
 'suggest': 2,
 'susceptibility': 1,
 'symptoms': 3,
 'target': 1,
 'therapeutic': 1,
 't

In [19]:
docs = gl.SArray(new_docs)

In [31]:
%%time
topic_model = gl.topic_model.create(docs, num_topics=10, num_iterations=3000)

Learning a topic model

Number of documents      5713

Vocabulary size     15418

Running collapsed Gibbs sampling

+-----------+---------------+----------------+-----------------+

...

| 2960      | 2m 21s        | 7.55602e+06    | 0               |

| 2970      | 2m 22s        | 7.42704e+06    | 0               |

| 2980      | 2m 22s        | 7.27794e+06    | 0               |

| 2990      | 2m 23s        | 7.11927e+06    | 0               |

| 3000      | 2m 23s        | 7.52761e+06    | 0               |

In [32]:
pyLDAvis.graphlab.prepare(topic_model, docs)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      13.353803        1       1  0.266831 -0.100559
2      10.333731        1       2 -0.135304  0.164809
8      10.208770        1       3 -0.133997 -0.090660
9       9.982341        1       4 -0.124429  0.057452
6       9.917930        1       5  0.221159 -0.088526
3       9.855992        1       6 -0.015263  0.003869
4       9.594482        1       7 -0.038977 -0.132916
5       9.229588        1       8 -0.149165 -0.015077
1       8.859152        1       9 -0.069076 -0.082529
7       8.664210        1      10  0.178221  0.284137, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
7682   Default  4408.000000         cells  4408.000000  30.0000  30.0000
14869  Default  8407.000000      patients  8407.000000  29.0000  29.0000
9322   Default  2470.000000        cancer  2470.000000  28.0000  28.0000
8982   Default  1888.000000         group  1888.000000  27.0000  27.0000
14055  Default  2427.000000      activity  2427.000000  26.0000  26.0000
7634   Default  1501.000000      receptor  1501.000000  25.0000  25.0000
9482   Default  1387.000000    inhibition  1387.000000  24.0000  24.0000
8058   Default  1799.000000   combination  1799.000000  23.0000  23.0000
266    Default  4970.000000       effects  4970.000000  22.0000  22.0000
9858   Default  1318.000000          days  1318.000000  21.0000  21.0000
3615   Default   922.000000      diabetes   922.000000  20.0000  20.0000
8432   Default  4493.000000         study  4493.000000  19.0000  19.0000
13237  Default  1076.000000          case  1076.000000  18.0000  18.0000
5628   Default  4594.000000     treatment  4594.000000  17.0000  17.0000
7520   Default  1846.000000          dose  1846.000000  16.0000  16.0000
1799   Default  1452.000000         tumor  1452.000000  15.0000  15.0000
14423  Default  1741.000000    associated  1741.000000  14.0000  14.0000
1688   Default  2400.000000       therapy  2400.000000  13.0000  13.0000
9535   Default  1188.000000        trials  1188.000000  12.0000  12.0000
4445   Default  1598.000000         human  1598.000000  11.0000  11.0000
11629  Default  1183.000000          risk  1183.000000  10.0000  10.0000
5689   Default   758.000000     hepatitis   758.000000   9.0000   9.0000
3486   Default  2242.000000          drug  2242.000000   8.0000   8.0000
11099  Default  2147.000000       disease  2147.000000   7.0000   7.0000
8937   Default  2082.000000           may  2082.000000   6.0000   6.0000
586    Default  1276.000000      reported  1276.000000   5.0000   5.0000
1566   Default   975.000000    expression   975.000000   4.0000   4.0000
12090  Default   689.000000  hypertension   689.000000   3.0000   3.0000
10447  Default   710.000000     metabolic   710.000000   2.0000   2.0000
6969   Default  1519.000000     inhibitor  1519.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
5767   Topic10   109.505001      allergic   111.370368   2.4291  -6.0554
375    Topic10   224.931903      seizures   258.920952   2.3052  -5.3356
6691   Topic10   427.681767          rats   587.803455   2.1280  -4.6930
3857   Topic10   300.210318      exposure   390.226852   2.1837  -5.0469
1530   Topic10   203.853947         sites   246.203243   2.2572  -5.4340
6726   Topic10   298.202894     injection   402.296604   2.1466  -5.0536
11239  Topic10   191.809401  inflammatory   245.564544   2.1989  -5.4949
3017   Topic10   192.813113          skin   257.360440   2.1572  -5.4897
644    Topic10   360.433050       calcium   662.304436   1.8376  -4.8641
3151   Topic10   308.240016      produced   532.473798   1.8993  -5.0205
14537  Topic10   447.756011          acid   997.351291   1.6451  -4.6472
5863   Topic10   200.842811  pretreatment   290.466090   2.0770  -5.4489
3288   Topic10   371.473

![title](images/graphLab_image.png)

** Top 3 Topics:**
- Cell cancer
- Diabetes effects
- High blood pressure

Even though GraphLab produces a very nice looking graph, my favorite Topic Modeler is LDA, because they make more sense.